<a href="https://colab.research.google.com/github/ajinkyabhanudas/SIADS696/blob/dev/Dataset_setup_Ajinkya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This notebook works on the first step of the train pipeline: data loading 
- The idea is to use the contents of this notebook as a makefile target so it can be the first step to run as we keep populating our datastore.








In [ ]:
# !git clone https://github.com/ajinkyabhanudas/SIADS696.git

In [1]:
import datetime
import enum
import glob
import json
import os
import re
from itertools import groupby

from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter(action='ignore')

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\poomk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\poomk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
try:
  os.chdir('SIADS696')
except:
  print("You're either already in the SIADS696 directory, or the path specified isn't accessible")

You're either already in the SIADS696 directory, or the path specified isn't accessible


### We'll extract our dataset in two parts:
- first extract the overall stats data per channel and store it along with the channel name instead of the id for available channels. (`overall_stats_raw_df`)
- And second, extract the per-video data into another dataframe. (`raw_df`)

In [4]:
rootdir = 'data'
feature_list = ['publishedAt', 'title', 'channelId', 'description',
                'channelTitle', 'tags', 'categoryId', 'viewCount', 
                'likeCount', 'favoriteCount', 'commentCount', 'duration',
                'definition', 'contentRating', 'topicCategories']

ov_dict_list = []
ov_temp_dict = {}

dict_list = []
temp_dict = {}

for path in glob.glob(f'./{rootdir}/*/*'):
  try:
    with open(path, "r") as read_file:
      data = json.load(read_file)
    for channel_id in data.keys():
      ov_temp_dict = data[channel_id]["channel_statistics"]
      channel_name = list(data[channel_id]["video_data"].keys())[0]
      ov_temp_dict["channelName"] = data[channel_id]["video_data"][channel_name]["channelTitle"]
      ov_dict_list.append(ov_temp_dict)
      
      for video_info in data[channel_id]["video_data"].values():
        temp_dict = video_info
        dict_list.append(temp_dict)       
  except:
    pass

overall_stats_raw_df = pd.DataFrame(ov_dict_list)
overall_stats_raw_df.drop("hiddenSubscriberCount", axis=1, inplace=True)

raw_df = pd.DataFrame(dict_list)
raw_df = raw_df[feature_list]

### Next, let's do a quick check on the data and take a quick look at the data in both the dfs

In [5]:
raw_df.info()
raw_df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55337 entries, 0 to 55336
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   publishedAt      55337 non-null  object
 1   title            55337 non-null  object
 2   channelId        55124 non-null  object
 3   description      55124 non-null  object
 4   channelTitle     55124 non-null  object
 5   tags             51972 non-null  object
 6   categoryId       55124 non-null  object
 7   viewCount        55050 non-null  object
 8   likeCount        54589 non-null  object
 9   favoriteCount    55121 non-null  object
 10  commentCount     54549 non-null  object
 11  duration         55125 non-null  object
 12  definition       55125 non-null  object
 13  contentRating    55125 non-null  object
 14  topicCategories  54331 non-null  object
dtypes: object(15)
memory usage: 6.3+ MB


,publishedAt,title,channelId,description,channelTitle,tags,categoryId,viewCount,likeCount,favoriteCount,commentCount,duration,definition,contentRating,topicCategories
46681,2022-04-12T12:30:04Z,Best Winter Photography Locations in the Magda...,UCKCRueDLpzx1vBcQHCc4ZHA,"If you're a photographer, videographer, or dro...",Going Awesome Places,"[Going Awesome Places, goingawesomeplaces, ile...",19,301,14,0,1,PT8M9S,hd,{},"[https://en.wikipedia.org/wiki/Hobby, https://..."
51854,2020-04-20T13:00:15Z,Salute your Core - 10 minute Vinyasa yoga with...,UCFYsO0t3zj0eJ_NcOlowTSA,In this short core sun salutation you connect ...,EkhartYoga,"[yoga, Online yoga, Online yoga classes, Yoga ...",26,2994,107,0,3,PT7M26S,hd,{},[https://en.wikipedia.org/wiki/Lifestyle_(soci...
50626,2020-09-19T14:00:07Z,50 Min Yoga for Athletes on Recovery Days | Br...,UCbfPq-uRqonJQli41muSLeQ,Welcome to this accessible yoga for beginners ...,Breathe and Flow,"[yoga for beginners, breathe and flow, yoga cl...",26,75840,1749,0,88,PT48M9S,hd,{},[https://en.wikipedia.org/wiki/Lifestyle_(soci...
6392,2015-12-22T23:33:02Z,Sticky Toffee Pudding,UCxD2E-bVoUbaVFL0Q3PvJTg,Sticky Toffee Pudding: a decadent holiday show...,ChefSteps,"[toffee pudding, british dessert, egg, cake, d...",26,219792,3099,0,132,PT3M23S,hd,{},"[https://en.wikipedia.org/wiki/Food, https://e..."
41846,2019-08-01T16:34:16Z,"Einstein's Biggest Blunder, Explained",UCUHW94eEFW7hkUMVaZz4eDg,Thanks to Brilliant for supporting MinutePhysi...,minutephysics,"[physics, minutephysics, science]",28,782490,31117,0,1346,PT6M4S,hd,{},[https://en.wikipedia.org/wiki/Knowledge]


In [6]:
overall_stats_raw_df.info()
overall_stats_raw_df.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   viewCount        149 non-null    object
 1   subscriberCount  149 non-null    object
 2   videoCount       149 non-null    object
 3   channelName      149 non-null    object
dtypes: object(4)
memory usage: 4.8+ KB


,viewCount,subscriberCount,videoCount,channelName
106,349786192,2040000,480,Beyond Science
83,10576768366,32900000,1141,NoCopyrightSounds
8,2598806060,17300000,189,Liza Koshy
66,172904891,1430000,88,History Buffs
10,3898272145,14400000,464,Rclbeauty101


### Let's examine our datatypes and convert them appropriately

In [7]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55337 entries, 0 to 55336
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   publishedAt      55337 non-null  object
 1   title            55337 non-null  object
 2   channelId        55124 non-null  object
 3   description      55124 non-null  object
 4   channelTitle     55124 non-null  object
 5   tags             51972 non-null  object
 6   categoryId       55124 non-null  object
 7   viewCount        55050 non-null  object
 8   likeCount        54589 non-null  object
 9   favoriteCount    55121 non-null  object
 10  commentCount     54549 non-null  object
 11  duration         55125 non-null  object
 12  definition       55125 non-null  object
 13  contentRating    55125 non-null  object
 14  topicCategories  54331 non-null  object
dtypes: object(15)
memory usage: 6.3+ MB


In [8]:
def duration_split(duration):
    try:
      for _, v in groupby(duration, str.isalpha):
        yield ''.join(v)
    except:
      yield np.nan
    
    
def duration_2_secs(duration, duration_split=duration_split):
  temp = 0
  # value = (duration_split(duration))
  for i, val in enumerate(duration_split(duration)):
    
    if len(str(duration)) <= 8:
      if i==1 and not val.isalpha():
        temp += float(val)*60
      
      if i==3 and not val.isalpha():
        temp += float(val)
    else:
      if i==1 and not val.isalpha():
        temp += float(val)*60*60
      
      if i==3 and not val.isalpha():
        temp += float(val)*60
      
      if i==5 and not val.isalpha():
        temp += float(val)

  return(temp)


# duration_2_secs("HR1PT51M12S")


In [9]:
def topic_extract(links_list):
  
  topics_list = []  
  try:
    for link in links_list:
      topics_list.append(link.split("/")[-1].lower())
    
    return(topics_list)
  except:
    pass

In [10]:
def text_prep(val):
  '''the goal is to replace the hypertexts in the 
  in any field to redundant names as vectorizing
  them could be misleading and also leak data'''

  val = str(val).lower()
  process_desc = re.sub(r'http[s]*:.*\w', 'url', val)
  process_desc = re.sub('[^a-zA-Z]', ' ', process_desc)
  process_desc = re.sub(r'\s+', ' ', process_desc)

  process_desc = nltk.sent_tokenize(process_desc)
  if not process_desc:
    process_desc = [nltk.word_tokenize(word) for word in process_desc]
  else:
    process_desc = [nltk.word_tokenize(word) for word in process_desc][0]
    
  
  word_list = [word for word in process_desc if word not in stopwords.words('english')]


  return word_list



In [11]:
text_prep("i if happening")

['happening']

In [12]:
def create_dataset(df, featureset, primary=True):

  if primary:
    df["publishedAt"] = pd.to_datetime(df.publishedAt)
    df['publishedDayNum'] = df.publishedAt.apply(lambda x: x.timetuple().tm_yday)
    df['publishedDayDelta'] = abs(df['publishedDayNum']-datetime.datetime.now().timetuple().tm_yday)
    df["categoryId"] = df.categoryId.astype(float)
    df["viewCount"] = df.viewCount.astype(float)
    df["likeCount"] = df.likeCount.astype(float)
    df["favoriteCount"] = df.favoriteCount.astype(float)
    df["commentCount"] = df.commentCount.astype(float)
    df['duration_secs'] = df.duration.apply(lambda x: duration_2_secs(x))
    df['topicCategories'] = df.topicCategories.apply(lambda x: topic_extract(x))
    df['tags'] = df.tags.apply(lambda x: topic_extract(x))
    df['channelTitle'] = df.channelTitle.str.lower()
    df['description'] = df.description.apply(lambda x: text_prep(x))
    df['title'] = df.title.apply(lambda x: text_prep(x))
    df['log_duration_secs'] = np.log(df.duration_secs+1)
  
  else:
    df['ov_viewCount'] = df.viewCount.astype(float)
    df['ov_subscriberCount'] = df.subscriberCount.astype(float)
    df['ov_videoCount'] = df.videoCount.astype(float)
    df['channelName'] = df.channelName.str.lower()

  return df[featureset]

- It's best to drop tags because, in many cases, the tags tend to contain the channel names, which could as we speculated earlier, lead to data leaks.
- We'll also drop channelId because it is visually unintuitve and we can encode the channelNames with simple Id's if we needed to.
- since we don't have data in the order of millions, we'd abstract out the publishing data to a new variable of `publishedDayNum` that contains the day of the year the video was published.
- on examining `contentRating` values, the dataset in non-null but majorly contains empty dictionaries, which makes it misleading. As a result, we'll drop this feature too.
- on examining `favoriteCount` values, the dataset in non-null but majorly contains a value of `0`, which makes it misleading. As a result, we'll drop it.

- dropped columns list [`tags`, `channelId`, `contentRating`] and [`publishedAt`] after deriving the `publishedDayNum` column from it.

In [13]:
features = ['title', 'description', 'channelTitle',
 'categoryId', 'viewCount', 'likeCount',
 'commentCount', 'duration_secs', 'log_duration_secs',
 'definition', 'topicCategories', 'publishedDayNum',
 'publishedDayDelta']

preproc1_df = create_dataset(raw_df, features)

KeyboardInterrupt: 

In [ ]:
preproc1_df.sample(5)

In [ ]:
preproc1_df.info()

Let's again take a look at what percentage of data is missing across all the fields.

In [ ]:
missing_data_perc_cols = (preproc1_df.isna().sum()/len(preproc1_df))*100
missing_data_perc_cols

In [ ]:
lost_rows_perc = len(preproc1_df.dropna())/len(preproc1_df)*100
lost_rows_perc

It's under 5% in across all cases, which isn't a bad thing at all and we can easily consider dropping the missing fields as we would regularly keep enriching our datastore, which would easily compensate for the missing 5% over time.

In [ ]:
preproc2_df = preproc1_df.dropna()

In [ ]:
preproc2_df.isna().sum()

In [ ]:
preproc2_df.drop('categoryId', axis=1).describe()

In [ ]:
preproc2_df.drop('categoryId', axis=1).corr()

In [ ]:
preproc2_df.drop('categoryId', axis=1).hist(figsize=(8,8));

We could consider applying a transformation to the data, to help address the heavy left-skew on most numerical features.

Let's quickly set the same steps up for the `overall_stats_raw_df` df.

In [ ]:
overall_stats_raw_df.head()

In [ ]:
overall_stats_raw_df.dtypes

In [ ]:
ov_features = ['channelName', 'ov_viewCount', 'ov_subscriberCount', 'ov_videoCount']
ov_stats_pre_df1 = create_dataset(overall_stats_raw_df, ov_features, primary=False)

In [ ]:
ov_stats_pre_df1.head()

In [ ]:
ov_stats_pre_df1.isna().sum()

With both the datasets ready and a basic relation overview amongst the features, we can now move onto the phase of text processing to make it machine interpretable.

In [ ]:
df = pd.merge(preproc2_df, ov_stats_pre_df1, how='left', left_on='channelTitle', right_on='channelName')
df.head()

In [ ]:
df['std_viewCount'] = df.groupby('channelTitle').viewCount.apply(lambda x: (x - x.mean()) / x.std()).values
# basic_model_data['log_duration_secs'] = np.log(basic_model_data.duration_secs+1)


In [ ]:
unique_topics_list = set()
for topic_list in df.topicCategories.values:
  for topic in topic_list:
    unique_topics_list.add(topic)

unique_titles_word_list = set()
for topic_list in df.title.values:
  for topic in topic_list:
    unique_titles_word_list.add(topic)

In [ ]:
# unique_titles_word_list

In [ ]:
basic_model_features = ['title', 'topicCategories', 'definition', 'publishedDayNum', 'log_duration_secs', 'std_viewCount', 'publishedDayDelta']
basic_model_data = df[basic_model_features]

basic_model_data.head()

In [ ]:
import seaborn as sns
#b4 transform
sns.distplot(basic_model_data.log_duration_secs+1);

In [ ]:
#after transform
# sns.distplot(np.log(basic_model_data.duration_secs+1));

In [ ]:
# sns.distplot(basic_model_data.publishedDayDelta);

In [ ]:
word2vec_titles = Word2Vec(basic_model_data.title, min_count=1)
word2vec_topics = Word2Vec(basic_model_data.topicCategories, min_count=1)

In [ ]:
# word2vec_titles.wv.vocab
word2vec_titles.wv['artificial'].shape

In [ ]:
# word2vec_titles.most_similar('travel')

In [ ]:
def w2v_models(df, col_list):

  model_dict = {}
  for col in col_list:
    model_dict[col] = Word2Vec(df[col], min_count=1)
  
  return model_dict

In [ ]:
w2v_model_dict = w2v_models(basic_model_data, ['title', 'topicCategories'])

In [ ]:
w2v_model_dict

In [ ]:
def vectorise_text(df, model):
  arr = np.zeros(100)
  for word in df:
    arr+=model.wv[word]
    
  return(np.mean(np.expand_dims(arr, axis=1), axis=1))

In [ ]:
# vectorise_text(["science", "art"], w2v_model_dict['title'])
basic_model_data.columns

In [ ]:
basic_model_data['vec_title'] = basic_model_data.title.apply(vectorise_text, model=w2v_model_dict['title'])
basic_model_data['vec_topicCategories'] = basic_model_data.topicCategories.apply(vectorise_text, model=w2v_model_dict['topicCategories'])

In [ ]:
['title', 'topicCategories', 'definition', 'publishedDayNum', 'log_duration_secs', 'std_viewCount', 'publishedDayDelta']

In [ ]:
titles = pd.DataFrame(basic_model_data.vec_title.tolist(), index= basic_model_data.index, columns=["title_"+str(i) for i in range(1,101)])
topics = pd.DataFrame(basic_model_data.vec_topicCategories.tolist(), index= basic_model_data.index, columns=["topics_"+str(i) for i in range(1,101)])


In [ ]:
data_df = pd.concat([titles, topics, basic_model_data[['publishedDayNum', 'log_duration_secs', 'std_viewCount', 'publishedDayDelta']], pd.get_dummies(basic_model_data.definition)], axis=1)

In [ ]:
data_df.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
data_train, data_test = train_test_split(data_df, test_size=0.20, random_state=42)

In [ ]:
train_X = data_train.drop("std_viewCount", axis=1)
train_y = data_train.std_viewCount

test_X = data_test.drop("std_viewCount", axis=1)
test_y = data_test.std_viewCount

In [ ]:
# train_X

In [ ]:
sc = StandardScaler()
train_X = sc.fit_transform(train_X)
test_X = sc.transform(test_X)



In [ ]:
pca = PCA(n_components=20)
pca.fit(train_X)
# print(()))

In [ ]:
import matplotlib.pyplot as plt
##SCREE PLOT
plt.plot(np.linspace(1,21,20),pca.explained_variance_ratio_);

In [ ]:
pca = PCA(n_components=5) #components based on the scree plot above

pca_train_X = pca.fit_transform(train_X)

plt.scatter(pca_train_X[:,0], train_y);

based on the first PCA, it is clear that the data is not linear and we would have to bear that in mind with the choice of our models

In [ ]:
pca_train_X.shape, train_y.shape

In [ ]:
pca = PCA(n_components=5) #components based on the scree plot above

pca_train_X = pca.fit_transform(train_X)
pca_test_X = pca.transform(test_X)


In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.dummy import DummyRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures

from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate


In [ ]:
train_X=pca_train_X
test_X=pca_test_X

In [ ]:
# data_train.sample(frac=0.2,replace=True)

In [ ]:
x_poly_train = PolynomialFeatures(3).fit_transform(train_X)
x_poly_test = PolynomialFeatures(3).transform(test_X)

train_X = x_poly_train
test_X = x_poly_test

In [ ]:
lr = LinearRegression(fit_intercept=False)
llr = Lasso(alpha=0.1)
rr = Ridge(alpha=1.0)
dr = DummyRegressor(strategy="mean")
svr = SVR(kernel = 'linear')
dtr = DecisionTreeRegressor(max_depth=2)

lr.fit(train_X, train_y)
llr.fit(train_X, train_y)
rr.fit(train_X, train_y)
dr.fit(train_X, train_y)
# svr.fit(train_X, train_y)
dtr.fit(train_X, train_y)

In [ ]:
lr.score(train_X, train_y)
# cross_val_score(dtr, train_X, train_y, cv=5)

In [ ]:
lr_pred = lr.predict(test_X)
dr_pred = dr.predict(test_X)
rr_pred = rr.predict(test_X)
# svr_pred = svr.predict(test_X)
dtr_pred = dtr.predict(test_X)
llr_pred = llr.predict(test_X)

In [ ]:
def ret_metrics(test_y, pred):

  mae = metrics.mean_absolute_error(test_y, pred)
  mse = metrics.mean_squared_error(test_y, pred)
  rmse = np.sqrt(mse)

  return(mae, mse, rmse)

In [ ]:
print(f"LR metrics for mae, mse, rmse are: {ret_metrics(test_y, lr_pred)}")
print(f"LLR metrics for mae, mse, rmse are: {ret_metrics(test_y, llr_pred)}")
print(f"DR metrics for mae, mse, rmse are: {ret_metrics(test_y, dr_pred)}")
print(f"RR metrics for mae, mse, rmse are: {ret_metrics(test_y, rr_pred)}")
# print(f"SVR metrics for mae, mse, rmse are: {ret_metrics(test_y, svr_pred)}")
print(f"DTR metrics for mae, mse, rmse are: {ret_metrics(test_y, dtr_pred)}")


In [ ]:
preds_df = pd.DataFrame({"true":test_y, "lr":lr_pred, "dr":dr_pred, "rr":rr_pred, "svr":-1, "dtr":dtr_pred, "llr":llr_pred })

In [ ]:
preds_df

In [ ]:
lists = sorted(zip(*[test_y, dtr_pred]))
sorted_x_test, sorted_y_test_pred = list(zip(*lists))
pca_n = 1

plt.plot(pca_train_X[:,pca_n],train_y,'.',color="darkblue", alpha=0.3,label="Train data")
plt.plot(pca_test_X[:,pca_n],test_y,'.',color='darkgreen', alpha=0.3, label="Test data")
plt.ylabel('Relative Title Quality')
plt.xlabel('PCA componenet')

plt.plot(sorted_x_test,sorted_y_test_pred,color='#FD8E70',linewidth=3, label="Best model")

plt.legend()
plt.show()